"""
Mask R-CNN wird mit Tipvortexcavitationdatensatz trainiert. Farbspritzereffekt(color splash effect) wird im Algorithmus Implementiert

------------------------------------------------------------

# Ein neues Modell wird mit den vortrainierten Coco-Gewichten trainiert
python3 Tipvortexcavitation.py train --dataset=/path/to/Tipvortexcavitation/dataset --weights=coco 
für Linux der Pfad und die Gewichte von Coco dataset.

python  Tipvortexcavitation.py train --dataset=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit- Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/datasets/Tipvortexcavitation --weights=coco

# Fortsetzen des Trainierens von einem Modell, was schon trainiert wurde 
python3 Tipvortexcavitation.py train --dataset=/path/to/Tipvortexcavitation/dataset --weights=last für Linux 

python  Tipvortexcavitation.py train --dataset=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit- Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/datasets/Tipvortexcavitation --weights=last

# Ein neues Modell wird mit den ImageNet-Gewichten trainiert
python3 Tipvortexcavitation.py train --dataset=/path/to/Tipvortexcavitation/dataset --weights=imagenet für Linux

python  Tipvortexcavitation.py train --dataset=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit- Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/datasets/Tipvortexcavitation --weights=imagenet
    
# Farbspritzer(color splash) auf ein Bild anwenden
python3 Tipvortexcavitation.py splash --weights=/path/to/weights/file.h5 --image=<URL or path to file> für Linux
    
python Tipvortexcavitation.py  splash --weights=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit- Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/mask_rcnn_tipvortexcavitation_0010.h5 --image=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit-Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/111.jpg
 
 # Farbspritzer (color splash) auf ein Video mit Gewichten von der Logsdatei anwenden
python3 Tipvortexcavitation.py splash --weights=last --video=<URL or path to file> für Linux
    
python  Tipvortexcavitation.py splash --weights=last --video=<URL or path to file>  für Windows
    
python Tipvortexcavitation.py splash --weights=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit- Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/mask_rcnn_tipvortexcavitation_0010.h5 --video=C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit-Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN/11.mp4
"""

In [47]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

# der Pfad der Datei von dem Mask R-CNN Projekt 
ROOT_DIR = os.path.abspath("C:/Users/majd4/Desktop/Bachelorarbeit/Bachelor-Arbeit-Daten/MaskRCNNProjekt/MaskRCNN_2/Mask_RCNN")

# Maske RCNN importieren
sys.path.append(ROOT_DIR)  
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn import visualize

# Pfad zur Datei mit den trainierten Gewichten
# was vorher trainiert wurde, natürlich wird es nochmals benutzt, um ein neues Modell zu trainieren 
# es wird das neue Modell besser in der erkennung von Tipvortexcavitation
WEIGHTSS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_tipvortexcavitation_0010.h5")

# hier ist die Datei, wo die logs gespeichert wurden 
# Epochen von dem Training 
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

############################################################
#  Konfigurationen
############################################################

# Esi ist hilfreich diese Website zu lesen, wenn man Änderung auf den Code vornehmen möchte 
#  https://github.com/matterport/Mask_RCNN/wiki


In [48]:
class TipvortexcavitationConfig(Config):
    """Konfiguration für das trainieren mit dem Tipvortexcavitationdatensatz
    Abgeleitet von dem basic Config Klasse und werden einige Werte überschrieben 
    """
    # ANZAHL DER zu verwendenden GPUs. Wenn nur eine CPU verwendet wird, muss dies auf 1 gesetzt werden.
    # es ist abhängig von dem Prozessor und die Grafikkarte, die man hat 
    # hier Auf meinem Laptop nutze ich AMD Ryzen 7 3700U with Radeon Vega Mobile Gfx 2.30 GHz
    GPU_COUNT = 1
    
    #Die Konfiguration wird einen erkennbaren Namen gegeben 
    NAME = "Tipvortexcavitation"

    # verwende eine GPU mit 12 GB Speicher, die zwei Bilder aufnehmen kann.
    # Passe nach unten an, wenn eine kleinere GPU verwendet wird.
    IMAGES_PER_GPU = 1

    # Anzahl der Klassen (einschließlich Hintergrund)
    NUM_CLASSES = 1 + 1  # Background + Tipvortexcavitation
    
    # Anzahl der Farbkanäle pro Bild.  
    # RGB = 3 (Bilder nur mit Farbe), 
    # grayscale = 1 (Bilder nur Schwarz und Weiß), 
    # RGB-D = 4 ( Bilder werden von einer Speziellen Kamera aufgenommen. 
    # jedes Pixel im Bildem ist mit einem Intensitätswert verbunden)
    IMAGE_CHANNEL_COUNT = 3
    
    
    # Backbone Netzwerk die Architektur
    # Unterstützte Werte sind: resnet50, resnet101.
    BACKBONE = "resnet101"

    # Anzahl der Trainingsschritte pro Epoche
    STEPS_PER_EPOCH = 10
    
    # Anzahl der Validierungsschritte, die am Ende jeder Trainingsepoche ausgeführt werden.
    
    VALIDATION_STEPS = 50

In [49]:
class TipvortexcavitationDataset(utils.Dataset):

    def load_Tipvortexcavitation(self, dataset_dir, subset):
        """Lade eine Teilmenge des Tip Vortex Cavitation-Datensatzes.
        """
        # Klassen hinzufügen.  
        self.add_class("Tipvortexcavitation", 1, "Tipvortexcavitation")

        # Trainings- oder Validierungsdatensatz?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

       
        annotations = json.load(open(os.path.join(dataset_dir, "Tipvortexcavitation_json.json")))
        annotations = list(annotations.values())  # 

        
        # Annotationen.unannotierte Bilder überspringen.
        annotations = [a for a in annotations if a['regions']]

        # Bilder hinzufügen
        for a in annotations:
            # Erhalte die x, y Koordinaten der Punkte der Polygone
            # Die Polygone werden in the shape_attributes gespeichert
            if type(a['regions']) is dict: #annotated
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else: # unanotated
                polygons = [r['shape_attributes'] for r in a['regions']] 

            # load_mask() benötigt die Bildgröße, um Polygone in Masken zu konvertieren.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "Tipvortexcavitation",
                image_id=a['filename'],  
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Erzeuge Instanz  für ein Bild.
       zurückgegeben:
        
        """
        
        image_info = self.image_info[image_id]
        if image_info["source"] != "Tipvortexcavitation":
            return super(self.__class__, self).load_mask(image_id)

        # Erzeuge Polygone zum einem bitmap mask of shape
        # [Höhe, Breite, Instanz_Anzahl]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        
       
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Gibt den Pfad des Bildes zurück."""
        info = self.image_info[image_id]
        if info["source"] == "Tipvortexcavitation":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [50]:
def train(model):
    """ das Modell trainieren."""
    # Trainingsdatensatz(Trainingsdatenmenge).
    dataset_train = TipvortexcavitationDataset()
    dataset_train.load_Tipvortexcavitation(args.dataset, "train")
    dataset_train.prepare()

    # Validierungsdatensatz(Validierungsdatenmenge)
    dataset_val = TipvortexcavitationDataset()
    dataset_val.load_Tipvortexcavitation(args.dataset, "val")
    dataset_val.prepare()

    
    
    
    
    # keine Notwendigkeit, alle Schichten zu trainieren. 
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='all')

In [51]:
def color_splash(image, mask):
    """Farbspritzer-Effekt anwenden.
    Bild: RGB Bild [Höhe, Breite, 3]
    mask:Instanzsegmentierung mask [Höhe, Breite, Anzahl der Instanzen]

    es wird schwarz weißes Bild mit Farbe zurückgegeben 
    """
    
    # Erstelle ein schwarzweiße Kopie von dem Farbigen Bild.   
    gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image)) * 255
    # Kopiere die gefärbte Maske von dem Originalbild  
    if mask.shape[-1] > 0:
        mask = (np.sum(mask, -1, keepdims=True) >= 1)
        splash = np.where(mask, image, gray).astype(np.uint8)
    else:
        splash = gray.astype(np.uint8)
    return splash

In [52]:
def detect_and_color_splash(model, image_path=None, video_path=None):
    assert image_path or video_path

    # Bild oder Video?
    if image_path:
        
        # das Modell für erkennung ausführen und erzeuge Farbspritzereffekt(color splash effect)
        print("Running on {}".format(args.image))
        # Bild lesen
        image = skimage.io.imread(args.image)
        #  Objekte erkennen
        r = model.detect([image], verbose=1)[0]
        # Farbspritzer
        splash = color_splash(image, r['masks'])
        # Ausgabe speichern 
        file_name = "splash_{:%Y%m%dT%H%M%S}.png".format(datetime.datetime.now())
        skimage.io.imsave(file_name, splash)
    elif video_path:
        import cv2
        
        vcapture = cv2.VideoCapture(video_path)
        width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = vcapture.get(cv2.CAP_PROP_FPS)

        
        file_name = "splash_{:%Y%m%dT%H%M%S}.avi".format(datetime.datetime.now())
        vwriter = cv2.VideoWriter(file_name,
                                  cv2.VideoWriter_fourcc(*'MJPG'),
                                  fps, (width, height))

        count = 0
        success = True
        while success:
            print("frame: ", count)
            # Das nächste Bild lesen
            success, image = vcapture.read()
            if success:
                # OpenCV gibt die Bilder als BGR zurück und wandelt zu RGB um 
                image = image[..., ::-1]
                # Objekte erkennen
                r = model.detect([image], verbose=0)[0]
                # Farbspritzer(Farbe spritzen)
                splash = color_splash(image, r['masks'])
                # RGB -> BGR um ein Bild in einem Video zu speichern
                splash = splash[..., ::-1]
                # dem Videoschreiber ein Bild hinzufügen
                vwriter.write(splash)
                count += 1
        vwriter.release()
    print("Speichere zu ", file_name)
    



In [ ]:
############################################################
#  Trainieren
############################################################

if __name__ == '__main__':
    import argparse

    # 
    parser = argparse.ArgumentParser(
        description='Trainiere Mask R-CNN, um Tipvortexcavitation zu erkennen')
    parser.add_argument("command",
                        metavar="<command>",
                        help="'train' or 'splash'")
    parser.add_argument('--dataset', required=False,
                        metavar="/path/to/balloon/dataset/",
                        help='Directory of the Tipvortexcavitation dataset')
    parser.add_argument('--weights', required=True,
                        metavar="/path/to/weights.h5",
                        help="Path to weights .h5 file or 'coco'")
    parser.add_argument('--logs', required=False,
                        default=DEFAULT_LOGS_DIR,
                        metavar="/path/to/logs/",
                        help='Logs and checkpoints directory (default=logs/)')
    parser.add_argument('--image', required=False,
                        metavar="path or URL to image",
                        help='Image to apply the color splash effect on')
    parser.add_argument('--video', required=False,
                        metavar="path or URL to video",
                        help='Video to apply the color splash effect on')
    args = parser.parse_args()

    # Argumente validieren
    if args.command == "train":
        assert args.dataset, "Argument --dataset is required for training"
    elif args.command == "splash":
        assert args.image or args.video,\
               "Provide --image or --video to apply color splash"

    print("Weights: ", args.weights)
    print("Dataset: ", args.dataset)
    print("Logs: ", args.logs)

    # Konfigurationen
    if args.command == "train":
        config = TipvortexcavitationConfig()
    else:
        class InferenceConfig(TipvortexcavitationConfig):
            # setz die Batchgröße(batch size) auf 1.da wir führen Inferenz auf ein Bild nach dem anderen aus 
            # Batchgröße(Batch size). Batch size = GPU_COUNT * IMAGES_PER_GPU
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
        config = InferenceConfig()
    config.display()

    # ein Modell erzeugen 
    if args.command == "train":
        model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=args.logs)
    else:
        model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=args.logs)

    # Gewichte Datei zum Laden auswählen 
    if args.weights.lower() == "coco":
        weights_path = WEIGHTSS_PATH
        # Gewichte Datei herunterladen
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif args.weights.lower() == "last":
        # letzte trainierte Gewichte finden oder das letzte trainierte Modell mit den Gewichten finden
        weights_path = model.find_last()
    elif args.weights.lower() == "imagenet":
        # Das neue Modell wird mit den Gewichten von Imagenetmodell Initialisieren
        weights_path = model.get_imagenet_weights()
    else:
        weights_path = args.weights

    # Gewichte laden
    print("Gewichte laden ", weights_path)
    if args.weights.lower() == "coco":
        
        model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
    else:
        model.load_weights(weights_path, by_name=True)

    # Trainieren or evaluieren
    if args.command == "train":
        train(model)
    elif args.command == "splash":
        detect_and_color_splash(model, image_path=args.image,
                                video_path=args.video)
    else:
        print("'{}' ist nicht erkannt. "
              "benutze 'trainieren' oder 'splash'".format(args.command))
